In [1]:
from line_solver import *
import numpy as np

In [2]:
# Model creation
# model = JMT2LINE('example_openModel_3.jsimg')  # Alternative: load from JMT file
model = Network('myModel')

print(f"Created model: {model.get_name()}")

Created model: myModel


In [3]:
# Block 1: Network nodes
node = np.empty(5, dtype=object)
node[0] = Source(model, 'Source 1')
node[1] = Queue(model, 'Queue 1', SchedStrategy.PS)
node[2] = ClassSwitch(model, 'ClassSwitch 1')  # Class switching is embedded in the routing matrix P
node[3] = Sink(model, 'Sink 1')
node[4] = Queue(model, 'Queue 2', SchedStrategy.PS)

In [4]:
# Block 2: Job classes  
jobclass = np.empty(3, dtype=object)
jobclass[0] = OpenClass(model, 'Class A', 0)
jobclass[1] = OpenClass(model, 'Class B', 0)
jobclass[2] = OpenClass(model, 'Class C', 0)

In [5]:
# Arrival processes
node[0].set_arrival(jobclass[0], Exp.fitMean(0.500000))  # (Source 1, Class A)
node[0].set_arrival(jobclass[1], Exp.fitMean(1.000000))  # (Source 1, Class B)
node[0].set_arrival(jobclass[2], Disabled.getInstance())  # (Source 1, Class C) - no direct arrivals

In [6]:
# Service processes at Queue 1
node[1].set_service(jobclass[0], Exp.fitMean(0.200000))  # (Queue 1, Class A)
node[1].set_service(jobclass[1], Exp.fitMean(0.300000))  # (Queue 1, Class B)
node[1].set_service(jobclass[2], Exp.fitMean(0.333333))  # (Queue 1, Class C)

In [7]:
# Service processes at Queue 2
node[4].set_service(jobclass[0], Exp.fitMean(1.000000))  # (Queue 2, Class A)
node[4].set_service(jobclass[1], Exp.fitMean(1.000000))  # (Queue 2, Class B)
node[4].set_service(jobclass[2], Exp.fitMean(0.150000))  # (Queue 2, Class C)

In [8]:
# Block 3: Network topology and class switching
# Initialize class switching matrix (identity matrix - no class change at the switch node itself)
C = node[2].init_class_switch_matrix()
C = np.eye(len(jobclass))  # Use number of classes for identity matrix size
node[2].set_class_switching_matrix(C)

In [9]:
# Initialize routing matrix
P = model.init_routing_matrix()

# Routing for Class A (index 0)
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)  # (Source 1, Class A) -> (Queue 1, Class A)
P.set(jobclass[0], jobclass[0], node[1], node[2], 1.0)  # (Queue 1, Class A) -> (ClassSwitch 1, Class A)
P.set(jobclass[0], jobclass[2], node[2], node[4], 1.0)  # (ClassSwitch 1, Class A) -> (Queue 2, Class C) - class switch!
P.set(jobclass[2], jobclass[2], node[4], node[3], 1.0)  # (Queue 2, Class C) -> (Sink 1, Class C)

# Routing for Class B (index 1)
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)  # (Source 1, Class B) -> (Queue 1, Class B)
P.set(jobclass[1], jobclass[1], node[1], node[2], 1.0)  # (Queue 1, Class B) -> (ClassSwitch 1, Class B)
P.set(jobclass[1], jobclass[2], node[2], node[4], 1.0)  # (ClassSwitch 1, Class B) -> (Queue 2, Class C) - class switch!
P.set(jobclass[2], jobclass[2], node[4], node[3], 1.0)  # (Queue 2, Class C) -> (Sink 1, Class C)

# Routing for Class C (index 2) - no arrivals, but define internal routing
P.set(jobclass[2], jobclass[2], node[0], node[1], 1.0)  # (Source 1, Class C) -> (Queue 1, Class C) - not used
P.set(jobclass[2], jobclass[2], node[1], node[2], 1.0)  # (Queue 1, Class C) -> (ClassSwitch 1, Class C) - not used
P.set(jobclass[2], jobclass[2], node[2], node[4], 1.0)  # (ClassSwitch 1, Class C) -> (Queue 2, Class C)
P.set(jobclass[2], jobclass[2], node[4], node[3], 1.0)  # (Queue 2, Class C) -> (Sink 1, Class C)

model.link(P)

In [10]:
# Solver options
options = Solver.defaultOptions()
options['keep'] = True
options['verbose'] = 1
options['cutoff'] = np.array([[1,1,0],[3,3,0],[0,0,3]])  # 5 nodes x 3 classes cutoff matrix
options['seed'] = 23000
options['samples'] = int(1e5)

In [ ]:
# Aligned with JAR test scenarios for oqn_cs_routing
# JAR tests: CTMC(cutoff=[[1,1,0],[3,3,0],[0,0,3]]), MVA(), MAM(), NC(), Fluid(), 
#           JMT(seed=23000, samples=100000), SSA(seed=23000, samples=100000, @Disabled)

solver = np.array([], dtype=object)

# CTMC with specific cutoff matrix (matches JAR)  
solver = np.append(solver, SolverCTMC(model, cutoff=np.array([[1,1,0],[3,3,0],[0,0,3]])))

# MVA with default settings (matches JAR)
solver = np.append(solver, SolverMVA(model))

# MAM with default settings (matches JAR)
solver = np.append(solver, SolverMAM(model))

# NC with default settings (matches JAR)
solver = np.append(solver, SolverNC(model))

# Fluid with default settings (matches JAR)
solver = np.append(solver, SolverFluid(model))

# JMT with seed=23000, samples=100000 (matches JAR)
solver = np.append(solver, SolverJMT(model, seed=23000, samples=100000))

# SSA with seed=23000, samples=100000 (matches JAR, but disabled in JAR tests)
# Commented out per JAR @Disabled annotation
# solver = np.append(solver, SolverSSA(model, seed=23000, samples=100000))

In [12]:
# Execute all solvers and collect results
AvgTable = np.empty(len(solver), dtype=object)

for s in range(len(solver)):
    AvgTable[s] = solver[s].get_avg_table()

CTMC analysis [method: default, lang: java, env: 21.0.8] completed in 0.968439s.
    Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0  Source 1  Class A  0.0000  0.0000  0.0000  0.0000  0.0000  1.8159
1  Source 1  Class B  0.0000  0.0000  0.0000  0.0000  0.0000  0.9412
3   Queue 1  Class A  0.8844  0.3632  0.4870  0.3247  1.8159  1.8159
4   Queue 1  Class B  0.7109  0.2824  0.7553  0.2518  0.9412  0.9412
8   Queue 2  Class C  0.6183  0.4136  0.2243  0.2243  2.7571  2.7571
Fluid analysis [method: closing, lang: java, env: 21.0.8] completed in 10.183573s.
    Station JobClass  QLen  Util  RespT  ResidT  ArvR    Tput
0  Source 1  Class A  0.00  0.00   0.00  0.0000   0.0  2.0000
1  Source 1  Class B  0.00  0.00   0.00  0.0000   0.0  1.0000
3   Queue 1  Class A  0.40  0.40   0.20  0.1333   0.0  2.0000
4   Queue 1  Class B  0.30  0.30   0.30  0.1000   0.0  1.0000
8   Queue 2  Class C  0.45  0.45   0.15  0.1500   0.0  3.0001
MVA analysis [method: default(egflin), lang: java,